In [ ]:
# id: YK9LIw42c4ql85gJ3OZk
# key : jRaHLgtFgo
import urllib.request
import datetime
import json

clientId = 'YK9LIw42c4ql85gJ3OZk'
clientSecret = 'jRaHLgtFgo'

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", clientId)
    req.add_header("X-Naver-Client-Secret", clientSecret)
  
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print(f'[{datetime.datetime.now()}] Url request Success')
            return response.read().decode('utf-8')
    except:
        print('error')
        
    return None

def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = '/news.json'
    srcText = urllib.parse.quote(srcText)
    parameters = f'?query={srcText}&start={start}&display={display}'
    url = base + node + parameters
    print(url)
    reponseDecode = getRequestUrl(url)
    if reponseDecode == None:
        return None
    else:
        # json 문자열을 Python 객체로 
        return json.loads(reponseDecode)

def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    originallink = post['originallink']
    link = post['link']
    pubDate = post['pubDate']
    jsonResult.append({'cnt': cnt,
                       'title': title,
                       'description': description,
                       'originallink': originallink,
                       'link': link,
                       'pubDate': pubDate})

node = 'news'
srcText = '선거'
cnt = 0
jsonResult =[]

jsonResponse = getNaverSearch(node, srcText, 1, 100) 
total = jsonResponse['total']
print(total)

while( (jsonResponse != None) and (jsonResponse['display']  != 0 )):
    for post in jsonResponse['items']:
        cnt += 1
        getPostData(post, jsonResult, cnt)
        
    start = jsonResponse['start']+ jsonResponse['display']
    jsonResponse = getNaverSearch(node, srcText, start, 10)
    
with open(f'./json/{srcText}_naver_{node}.json', 'w', encoding='utf-8-sig') as outputFile:
    jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
    outputFile.write(jsonFile)
    

In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

path = 'D:\\program_files\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.implicitly_wait(2)
driver.get('https://www.youtube.com/c/paikscuisine/videos')

C:\Users\admin\AppData\Local\Temp\ipykernel_13460\3138949881.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [66]:
# ActionChains 를 사용하기 위해서.
from selenium.webdriver import ActionChains
import time
allVideos = driver.find_elements(By.ID, 'dismissible')
body_tag = driver.find_element(By.TAG_NAME, 'body')

while True:
    body_tag.send_keys(Keys.END)
    last_height = driver.execute_script('return document.documentElement.scrollHeight')
    print(lastHeight)
    
    for i in range(10):
        body_tag.send_keys(Keys.END)
        time.sleep(1)
    
    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    if new_height == last_height:
        break
    
   

21881
21881
21881


In [ ]:

# # id가 something 인 element 를 찾음
# some_tag = driver.find_element(By.ID,'show-more-button')

# # somthing element 까지 스크롤
# action = ActionChains(driver)
# action.move_to_element(some_tag).perform()

In [67]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
from bs4 import BeautifulSoup
import pymysql

dbURL = '127.0.0.1'
dbPort = 3306
dbUser = 'root'
dbPass = 'root'

conn = pymysql.connect(host = dbURL, 
                       port = dbPort, 
                       user = dbUser, 
                       passwd = dbPass,
                       db = 'bigdb',
                       charset = 'utf8',
                       use_unicode=True)



In [11]:
insert_weather = 'insert into forecast (city, tmef, wf, tmn, tmx) values (%s,%s,%s,%s,%s)'


req = requests.get('http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId2=108')
html = req.text
soup = BeautifulSoup(html, 'lxml')

weather = dict()

select_data = "select tmef from forecast order by tmef desc limit 1"
cur =conn.cursor()
cur.execute(select_data)
lastdata = cur.fetchone()
print(lastdata)

for i in soup.find_all('location'):
    weather[i.find('city').text] = []

    for j in i.find_all('data'):
        temp = []
        if (lastdata is None) or (j.find('tmef').text > str(lastdata[0]) )  :
            temp.append(j.find('tmef').text)
            temp.append(j.find('wf').text)
            temp.append(j.find('tmn').text)
            temp.append(j.find('tmx').text)
            weather[i.find('city').text].append(temp)
        

for i in weather:
    for j in weather[i]:
        cur = conn.cursor()
        cur.execute(insert_weather, (i, j[0],j[1], j[2], j[3]))
        conn.commit()


('2022-08-01 00:00',)


c:\Users\admin\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [19]:
select_busan = "select max(tmx), min(tmn) from forecast where city = '부산' order by tmef desc"
cur = conn.cursor()
cur.execute(select_busan)
busan = cur.fetchone()
print(busan)

('31', '24')
